In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tarfile
import os
import gzip
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data_base_path = os.path.join(os.pardir, 'data')
data_folder = 'lab-03-data'
tar_path = os.path.join(data_base_path, data_folder + '.tar.gz')
with tarfile.open(tar_path, mode='r:gz') as tar:
    tar.extractall(path=data_base_path)

In [3]:
def extract_data(filename, image_shape, image_number):
    with gzip.open(filename) as bytestream:
        bytestream.read(16)
        buf = bytestream.read(np.prod(image_shape) * image_number)
        data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
        data = data.reshape(image_number, image_shape[0], image_shape[1])
    return data


def extract_labels(filename, image_number):
    with gzip.open(filename) as bytestream:
        bytestream.read(8)
        buf = bytestream.read(1 * image_number)
        labels = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
    return labels

In [4]:
image_shape = (28, 28)
train_set_size = 60000
test_set_size = 10000

data_part2_folder = os.path.join(data_base_path, data_folder, 'part2')

train_images_path = os.path.join(data_part2_folder, 'train-images-idx3-ubyte.gz')
train_labels_path = os.path.join(data_part2_folder, 'train-labels-idx1-ubyte.gz')
test_images_path = os.path.join(data_part2_folder, 't10k-images-idx3-ubyte.gz')
test_labels_path = os.path.join(data_part2_folder, 't10k-labels-idx1-ubyte.gz')

train_images = extract_data(train_images_path, image_shape, train_set_size)
test_images = extract_data(test_images_path, image_shape, test_set_size)
train_labels = extract_labels(train_labels_path, train_set_size)
test_labels = extract_labels(test_labels_path, test_set_size)

In [5]:
X_train = np.array(train_images)
X_test = np.array(test_images)

N_train, w, h = X_train.shape
N_test,_,_ = X_test.shape

x_train = np.array(train_images).reshape(N_train,w,h,1)
x_test = np.array(test_images).reshape(N_test,w,h,1)

y_train = np.zeros((N_train,10))
for n, i in enumerate(train_labels):
    y_train[n][i] = 1
    
y_test = np.zeros((N_test,10))
for n, i in enumerate(test_labels):
    y_test[n][i] = 1

In [87]:
def train_model(x_train,y_train,x_test,y_test):
    
    IMG_HEIGHT = 28
    IMG_WIDTH = 28
    
    model = Sequential()

    model.add(layers.Conv2D(filters=6, kernel_size=(5, 5), activation='relu', input_shape=(28,28,1)))
    model.add(layers.AveragePooling2D())
    model.add(layers.Conv2D(filters=16, kernel_size=(5, 5), activation='relu'))
    model.add(layers.AveragePooling2D())
    model.add(layers.Conv2D(filters=120, kernel_size=(3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(units=84, activation='relu'))
    model.add(layers.Dense(units=10, activation = 'softmax'))
    
    model.compile(optimizer='sgd',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    datagen = ImageDataGenerator(rotation_range=360, zoom_range = 0.2)
    training_data = datagen.flow(x_train,y_train,batch_size = 100)
    testing_data = datagen.flow(x_test,y_test,batch_size = 100)
    
    epochs = 50
    history = model.fit(training_data, epochs=epochs, validation_data = testing_data)
    
    model.save('LeNet_Model')
    
    return model

In [88]:
model = train_model(x_train,y_train,x_test,y_test)

Train for 600 steps, validate for 100 steps
Epoch 1/50
600/600 [==============================] - 29s 48ms/step - loss: 1.5343 - accuracy: 0.4735 - val_loss: 0.7829 - val_accuracy: 0.7449
Epoch 2/50
600/600 [==============================] - 28s 46ms/step - loss: 0.6160 - accuracy: 0.8036 - val_loss: 0.4365 - val_accuracy: 0.8581
Epoch 3/50
600/600 [==============================] - 27s 46ms/step - loss: 0.4272 - accuracy: 0.8660 - val_loss: 0.3307 - val_accuracy: 0.8933
Epoch 4/50
600/600 [==============================] - 28s 46ms/step - loss: 0.3529 - accuracy: 0.8905 - val_loss: 0.2922 - val_accuracy: 0.9061
Epoch 5/50
600/600 [==============================] - 28s 46ms/step - loss: 0.3097 - accuracy: 0.9039 - val_loss: 0.2818 - val_accuracy: 0.9126
Epoch 6/50
600/600 [==============================] - 27s 45ms/step - loss: 0.2775 - accuracy: 0.9123 - val_loss: 0.2641 - val_accuracy: 0.9137
Epoch 7/50
600/600 [==============================] - 28s 47ms/step - loss: 0.2571 - accurac

In [102]:
np.unique(test_labels)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])